In [5]:
"""
Jason Stranne
"""
import numpy as np
import os
import sys
import gc
from Two_Combined_Downstream import DownstreamNet_Combined, Downstream_Dataset, print_class_counts, num_correct, reduce_dataset_size
from Two_Combined_Downstream import smallest_class_len, restrict_training_size_per_class, train_end_to_end
sys.path.insert(0, '..')
from Stager_net_pratice import StagerNet
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import LeaveOneGroupOut, GroupKFold
from torch import Tensor
from torch.utils.data import DataLoader, TensorDataset


In [6]:
def train_different_classes(model_patha, model_pathb, tset, vset, epochs, sample_list):
    outList=[]
    balanced_acc_out=[]
    for i in sample_list:
        acc, balanced_acc = train_end_to_end(model_patha, model_pathb, tset, vset, i, epochs, 3)
        outList.append(acc)
        balanced_acc_out.append(balanced_acc)
    return outList, balanced_acc_out

# Models

In [16]:
model_1 = "RP_stagernet_30_120.pth"
#model_2 = "TS_stagernet_30_30.pth"
model_2 = "CPC_stagernet_16.pth"

In [17]:
root = os.path.join("Mouse_Training_Data", "Windowed_Data", "")
datasets_list=[]
print('Loading Data')
f=open(os.path.join("training_names.txt"),'r')
lines = f.readlines()
x_vals = []
y_vals = []
groups = []
index = 0
for line in lines:
    recordName=line.strip()
    print('Processing', recordName)
    data_file=root+recordName+os.sep+recordName
    datasets_list.append(Downstream_Dataset(path=data_file))
    d = Downstream_Dataset(path=data_file)
    x_vals.append(d.data)
    y_vals.append(d.labels)
    groups.append(np.ones(len(d.labels))*index)
    index+=1
f.close()


x_vals = np.vstack(x_vals)
y_vals = np.concatenate(y_vals, axis=0)
groups = np.concatenate(groups, axis=0)
print(x_vals.shape)
print(y_vals.shape)
print(groups.shape)


# logo = LeaveOneGroupOut()
kfold = GroupKFold(n_splits=5)
# logo.get_n_splits(x_vals, y_vals, groups)
kfold.get_n_splits(x_vals, y_vals, groups)
                    
                    
result_dict = {}
# dtype=torch.int32
for train_index, test_index in kfold.split(x_vals, y_vals, groups):
    unique = np.unique(groups[test_index])
    # group_num = groups[test_index][0]
    print("Leaving out mouse number:", unique)
    training_set = TensorDataset(torch.tensor(x_vals[train_index], dtype=torch.float), torch.tensor(y_vals[train_index], dtype=torch.long))
    test_set = TensorDataset(torch.tensor(x_vals[test_index], dtype=torch.float), torch.tensor(y_vals[test_index], dtype=torch.long))
    print(len(training_set))
    smallest_class = smallest_class_len(training_set, 3)
    num_samples=[]
    temp=1
    while temp < smallest_class:
        num_samples.append(temp)
        temp*=10
    num_samples.append(None)
    accuracy, balanced_accuracy = train_different_classes(
        model_1,
        model_2,
        training_set, 
        test_set, 
        100, 
        num_samples
    )
    for num_pos, acc in zip(num_samples, accuracy):
        if num_pos not in result_dict:
            result_dict[num_pos] = []
        result_dict[num_pos].append(acc)
    print(result_dict)
    
for k in result_dict:
    result_dict[k] = np.mean(result_dict[k])
print(result_dict)



Loading Data
Processing MouseCKA1_030515_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKB9_022715_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKL1_062514_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKL5_063014_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKL7_063014_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
labels shape (400,)
data shape (400, 3000, 11)
removed 0 unknown entries
Processing MouseCKN1_063014_HCOFTS
labels shape (400,)
data shape (400, 3000, 11)


In [20]:
print("RP and CPC")

# {1: 0.6099666666666667, 10: 0.82415, 100: 0.8636166666666666, 1000: 0.9001999999999999, None: 0.9190666666666665}
# {1: 0.5544499999999999, 10: 0.8084833333333332, 100: 0.8641166666666666, 1000: 0.8998166666666666, None: 0.9183833333333334}
# {1: 0.5372833333333333, 10: 0.81735, 100: 0.8723333333333334, 1000: 0.90175, None: 0.9137833333333335}

trial1=[0.6099666666666667, 0.82415, 0.863616666666666, 0.9001999999999999, 0.9190666666666665]
trial2=[0.5544499999999999, 0.8084833333333332, 0.8641166666666666, 0.8998166666666666, 0.9183833333333334]
trial3=[0.5372833333333333, 0.81735, 0.8723333333333334, 0.90175, 0.9137833333333335]
avg=[]
for i in range(len(trial1)):
    avg.append((trial1[i]+trial2[i]+trial3[i])/3)
print(avg)

print("RP and TS")

trial1=[0.5429999999999999, 0.8103333333333333, 0.8850166666666667, 0.92965, 0.9447833333333333]
trial2=[0.6306166666666666, 0.8315166666666667, 0.8900333333333332, 0.9319333333333333, 0.94625]
trial3=[0.5587500000000001, 0.8241666666666667, 0.8928, 0.9307166666666667, 0.9464333333333332]
avg=[]
for i in range(len(trial1)):
    avg.append((trial1[i]+trial2[i]+trial3[i])/3)
print(avg)

print("TS and CPC")

trial1=[0.5960833333333333, 0.8314333333333334, 0.8869166666666667, 0.9306166666666666, 0.9405833333333333]
trial2=[0.5497, 0.8232333333333333,0.8875666666666666, 0.9355666666666668, 0.94615]
trial3=[0.5587500000000001, 0.8241666666666667, 0.8928, 0.9307166666666667, 0.9464333333333332]
avg=[]
for i in range(len(trial1)):
    avg.append((trial1[i]+trial2[i]+trial3[i])/3)
print(avg)


RP and CPC
[0.5672333333333333, 0.816661111111111, 0.8666888888888886, 0.900588888888889, 0.9170777777777778]
RP and TS
[0.5774555555555555, 0.8220055555555555, 0.8892833333333332, 0.9307666666666666, 0.9458222222222222]
TS and CPC
[0.5681777777777778, 0.8262777777777778, 0.8890944444444445, 0.9323, 0.9443888888888888]


In [ ]:
trial1=[0.5801833333333334, 0.7514333333333, 0.8498000000000001, 0.9018, 0.9062333333333333]
trial2=[0.55125, 0.72645, 0.8608833333333331, 0.8951666666666667, 0.9088833333333334]
trial3=[0.5833833333333334, 0.7462333333333333, 0.8501999999999998, 0.8996999999999999, 0.9077333333333332]
avg=[]
for i in range(len(trial1)):
    avg.append((trial1[i]+trial2[i]+trial3[i])/3)
print(avg)

In [4]:
smallest_class = smallest_class_len(training_set)
num_samples=[]
temp=1
while temp < smallest_class:
    num_samples.append(temp)
    temp*=10
num_samples.append(None)
print(num_samples)

[1, 10, 100, 1000, None]


For RP and TS:


{1: 0.5580871212121212, 10: 0.8284545454545456, 100: 0.8890378787878788, 1000: 0.9424356060606061, None: 0.9550568181818182}

[0.5580871212121212, 0.8284545454545456, 0.8890378787878788, 0.9424356060606061, 0.9550568181818182]

For RP and CPC:

{1: 0.6458068181818182, 10: 0.8400265151515152, 100: 0.8980492424242424, 1000: 0.921814393939394, None: 0.9393863636363635}

[0.6458068181818182, 0.8400265151515152, 0.8980492424242424, 0.921814393939394, 0.9393863636363635]


For TS and CPC:

{1: 0.6103068181818182, 10: 0.8269090909090909, 100: 0.8815113636363636, 1000: 0.9322386363636364, None: 0.9517613636363637}

[0.6103068181818182, 0.8269090909090909, 0.8815113636363636, 0.9322386363636364, 0.9517613636363637]

In [9]:
CPC_vals = train_different_classes("RP_stagernet.pth", "TS_stagernet.pth", "CPC_stagernet.pth", training_set, validation_set, 100, num_samples)

len of the dataloader is: 1
Start Training
[0.33246640658864324]
len of the dataloader is: 1
Start Training
[0.33246640658864324, 0.4729085392284352]
len of the dataloader is: 2
Start Training
[0.33246640658864324, 0.4729085392284352, 0.5831816211530125]
len of the dataloader is: 20
Start Training
[0.33246640658864324, 0.4729085392284352, 0.5831816211530125, 0.6476809709579541]
len of the dataloader is: 68
Start Training
[0.33246640658864324, 0.4729085392284352, 0.5831816211530125, 0.6476809709579541, 0.7046380580840919]


In [6]:
print(CPC_vals)

[0.28097095795405286, 0.42418725617685304, 0.5448634590377113, 0.5814477676636324, 0.6376246207195492]
